1. Google Drive 연결 & 경로 이동

In [ ]:
cd /content/drive/MyDrive/YOLOv3

2. Alexey/darknet git clone

In [ ]:
!git clone https://github.com/AlexeyAB/darknet.git

3. Custom YOLOv3-tiny
- yolov3-tiny_obj.cfg 파일 수정
- obj.data & obj.names 파일 생성
- obj image 파일 라벨링해서 zip
- https://ichi.pro/ko/sayongja-jijeong-yolov4-gaeche-tamjigi-gyoyug-google-colab-sayong-6710443722856
- https://codingzzangmimi.tistory.com/76


OPENCV 및 GPU 활성화

In [ ]:
%cd darknet/
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
!sed -i 's/LIBSO=0/LIBSO=1/' Makefile

In [ ]:
!make

관계없는 데이터 및 cfg 폴더 정리

In [ ]:
%cd data/
!find -maxdepth 1 -type f -exec rm -rf {} \;
%cd ..
%rm -rf cfg/
%mkdir cfg

In [ ]:
## obj.zip 파일 압축 해제 (1번만 실행해도 된다.)
!unzip /content/drive/MyDrive/YOLOv3/obj.zip -d data/

In [ ]:
# darknet/cfg
!cp /content/drive/MyDrive/YOLOv3/yolov3-tiny_obj.cfg cfg

# darknet/data
!cp /content/drive/MyDrive/YOLOv3/obj.names data
!cp /content/drive/MyDrive/YOLOv3/obj.data  data

In [ ]:
!cp /content/drive/MyDrive/YOLOv3/process.py .

In [ ]:
!python process.py

In [ ]:
!ls data/

사전 훈련된 YOLOv3 tiny 가중치 다운로드
- https://drive.google.com/file/d/18v36esoXCh-PsOKwyP2GWrpYDptDY8Zf/view


In [ ]:
# mAP 출력 버전 (Colab에서는 안 돌아간다..)
# !./darknet detector train data/obj.data cfg/yolov3-tiny_obj.cfg yolov3-tiny.conv.11 -map

In [ ]:
!./darknet detector train data/obj.data cfg/yolov3-tiny_obj.cfg yolov3-tiny.conv.11 -dont_show -map

중간에 런타임이 끊겼을 때:

In [ ]:
!./darknet detector train data/obj.data cfg/yolov4-custom.cfg /content/drive/MyDrive/YOLOv3/training/yolov3-tiny_last.weights -dont_show -map

Colab 연결 해지를 막는 자바 스크립트:  
> (Ctrl + Shift + i) 를 눌러 콘솔로 이동  
> 다음 코드를 붙여넣고 Enter

In [ ]:
function ClickConnect(){
console.log("Working"); 
document
  .querySelector('#top-toolbar > colab-connect-button')
  .shadowRoot.querySelector('#connect')
  .click() 
}
setInterval(ClickConnect,60000)

# 성능 확인

1. chart.png 파일 을 보면 훈련 된 모든 가중치의 성능을 확인할 수 있다 .

In [ ]:
import cv2
import matplotlib.pyplot as plt

def imShow(path):
  %matplotlib inline
  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)
  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

In [ ]:
imShow('chart.png')

2. MAP (평균 정밀도) 확인
- yolov3-custom_4000.weights, yolov4-custom_5000.weights, yolov4-custom_6000.weights 등에 대해 1000 번 반복 할 때마다 저장된 모든 가중치에 대해 mAP를 확인할 수 있다.
- 이렇게하면 어떤 가중치 파일이 최상의 결과를 제공하는지 찾을 수 있다.
- mAP가 높을수록 좋다.

In [ ]:
!./darknet detector map data/obj.data cfg/yolov3-tiny_obj.cfg /content/drive/MyDrive/YOLOv3/training/yolov3-tiny_obj_1000.weights -points 0

In [ ]:
!./darknet detector map data/obj.data cfg/yolov3-tiny_obj.cfg /content/drive/MyDrive/YOLOv3/training/yolov3-tiny_obj_last.weights -points 0

In [ ]:
!./darknet detector map data/obj.data cfg/yolov3-tiny_obj.cfg /content/drive/MyDrive/YOLOv3/training/yolov3-tiny_obj_best.weights -points 0

3. cfg 파일 Test 주석 처리 풀고 테스트하기

In [ ]:
%cd cfg
!sed -i 's/batch=64/batch=1/' yolov3-tiny_obj.cfg
!sed -i 's/subdivisions=16/subdivisions=1/' yolov3-tiny_obj.cfg
%cd ..